# Creating command line tools CLI

## The `click` module

For System Administrators, command line tools are the daily bread. This section helps to create beautiful command line interfaces in a composable way with as little code as necessary. The [click module](https://click.palletsprojects.com/en/7.x/api/#click.option) is the most popular among the many that exist, e.g. [argparse](https://click.palletsprojects.com/en/7.x/why/#why-not-argparse), [docopt](https://click.palletsprojects.com/en/7.x/why/#why-not-docopt-etc) among others.

The click module not only makes it relatively easy to create nice command line interfaces, but also implements many best practices. For example, it automatically creates a help page for you (for every command and subcommand), handles flags and passwords in a consistent and predictable way.

Installation with `pip` is easy. Note the `-U` parameter, a shorthand for `--upgrade` which forces pip to install the latest version of that module:

In [ ]:
!pip install -U click

A minimal example, taken from the [click documentation](https://click.palletsprojects.com/en/7.x/):

In [32]:
import click

@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo('Hello %s!' % name)


The first line we already understand:

```python
import click
```

This just imports the click module, ready to be used.

Next, we discover some new elements:

```python
@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name', help='The person to greet.')
def hello(count, name):
```

These elements are so called so called **decorators**. Their functionality is to **wrap** a function or a method. Before the `def hello()` function is being executed, first the `click.command()` is executed, then two times a `click.option()` decorator.

The options `--count` and `--name` are passed as parameters `count` and `name` to our `hello(count, name)` function.

### how to define the entry point in a script

If we put the code above into a python script (open [hello_click.py](hello_click.py)), and then we run it with the Python interpreter `python3 hello_click.py`, what happens?

**Nothing**. But why?

The reason is simple: The Python interpreter needs to know what to run. It imports the click module, then it compiles our `hello()` method, finally reaches the end of the script and exits.

We have to tell Python to execute our function if **the caller is the main process**. We could simply add

```python
hello()
```

at the end of our script, that function would **always be executed**, as shown in [hello_click_1.py](hello_click_1.py)

In [16]:
!./hello_click_1.py --name World --count 3

Hello World!
Hello World!
Hello World!


However, if we only want to import our `hello_click_1` script from some other script, e.g. [import_hello_click_1.py](import_hello_click_1.py), the function gets executed automatically:

In [17]:
!./import_hello_click_1.py

Klick on the stop button ∎ to interrupt the execution above.

To avoid this behaviour and to distinguish between the two cases, we need to **identify the caller**. In Python, this statement can typically be found at the end of the main script:

```python
if __name__ == '__main__':
    hello()
```

### Excercise 1

modify hello_click.py so you can

1. execute it from command line
2. import from within this Jupyter notebook

### Exercise 2

Have a look at [main.py](main.py), the entry point for the ethz-iam-webservice command line tool I wrote this year. Can you identify the entry point?

## Obtaining parameters

**Normal values**

As we have noticed already, we can obtain parameters from the command line via decorators, e.g.

```python
@click.option("--name", prompt="Your name", help="The person to greet.")
```

If we want to use this method in a «traditional» context, we would use `click.prompt() instead`:

In [29]:
import click

name = click.prompt(text="Your name")

print(f"Hello {name}!")

Your name: 

 gil


Hello gil!


**Boolean values: Flags**

Often we want to adjust the behaviour of our command line tool by providing boolean flags. The presence of the flag will set it to True:

```python

@click.option(
    '-d', '--delete',
    is_flag=True,
    help='delete this user'
)
```



**Passwords**

```python
@click.option(
    '--password', 
    prompt=True,                 # does what it says: it prompts, it actively waits for input
    confirmation_prompt=True,    # the password has to be entered twice
    hide_input=True              # do not show the input characters, of course...
)
```


### Exercise 3

Have again a look at [main.py](main.py). How and at when needs the password entered to be? What is the `ctx` object all about?

## Sub commands with click

Sub-commands are a way to conveniently group commands that have certain parameters in common. For example, the `ethz-iam-webservice` CLI offers the group subcommand:

```bash
$ iam group my_group -i
$ iam group my_group -a new_user1 -a new_user2 -r old_user3
$ iam group my_group --delete
```

Two steps are required:

1. decorate a function with `@click.command()`
2. register the command with the `click.add_command()` function

### Exercise 4

Investigate [main.py](main.py) and see how the subcommands are registered. Which other subcommands are registered?

## Outlook: how to configure your `setup.py` to register a command line utility automatically

When you install the `ethz-iam-webservice` utility using `pip install`, you will end up with a command line tool named `iam`. How did that happen?

When you run a `pip install`, the `setup.py` is executed. It contains a lot of options, among them the entry_points with the so called **console_scripts**:

```python
    entry_points={
        'console_scripts' : [
            'iam=ethz_iam_webservice.main:cli'
        ]
    }
```

The setuptools will create a small python script for you and store it in the `bin` directory next to your Python interpreter. That Python script has the name `iam`, imports the `cli` function from the `ethz_iam_webservice.main` module and executes it. Note that `ethz_iam_webservice.main` actually refers to `ethz_iam_webservice/main.py`.

### Exercise 5

Look at the full `setup.py` in the [nethz-iam-repository](https://gitlab.ethz.ch/vermeul/nethz-iam-webservice/-/blob/master/). Locally clone the repository and install the utility, using the `pip install` command.